## Introduction

While writing this notebook I will be reading the first chapter of the book and wold summarize some programming assignments, the algorithms, extra challenge problems that I attempt and the math behind the analysis of the algorithms explained.

---
 We will start by looking at the Integer Multiplication problem. 
 
To multiply two numbers we need to have the following primitive operations

- Add two single digit numbers
- Multiply two single-digit numbers
- Append of prepend a 0 to a number

Following image shows how perform the following multiplication $5678 \times 1234$
 
 ![Multiply](ConventionalMultiplication.png)
 
 Let us now analyze the total number of operations for multiplying two n digit numbers
 
 A partial product involves multiplying 1 digit of an n digit number with another n digit. Each multiplication involves atmost $2n$ operations where we have n multiplications for an n digit number and then an addition if a carry is generated from the previous multiplication operation.  Essentially we have $n \times 2n$ operations giving is $2n^2$ multiply operations for multiplying two n digit numbers.
 
 We still are left with adding up these partial products. We have n rows of these partial sums and each of these n rows can have a carry giving us a maximum of 2n operations per row and thus adding up all partial sums gives is $n \times 2n = 2n^2$ operations
 
 Thus the conventional multiplication takes $2n^2 + 2n^2 = 4n^2$ operations. Or, more generally the number of operations for performing multiplication of two n digit numbers is $Cn^ 2$ operations. Thus the work increases quadratically with the number of digits in the the number.
 
 ---
 
 ***Can we do better than $n^2$ complexity?***
 
 Let us look at an alternate way to multiply two numbers known as *Karatsuba Multiplication*. We will us ethe same two numbers 5678 and 1234 and see Karatsuba Multiplication in action.
 
 - Break up these two 4 digit numbers into halfs, thus we get 4 smaller numbers of 2 digits each. We call them a, b, c and d. Thus for the above example, 5678 will be split in two numbers 56 and 78 and we call them a and b respectively. Similarly, 1234 will be split into c and d with values 12 and 34 respectively.
 - Compute $a \times c$ which is $56 \times 12 = 672$
 - Compute $b \times d$ which is $78 \times 34 = 2652$
 - Compute $(a + b) \times (c + d)$ which is $(56 + 78) \times (12 + 34) = 6164$
 - Subtract the first two results from the third, thus we get $6164 - 672 - 2652 = 2840$
 - Compute $10^4 \times 672 + 10^2 \times  2840 + 2652 = 7006652$
 
 The above result is exactly same as the one we got using the conventional method.
 
 ---
 
 Before we implement the Karatsuba multiplication, let us implement a recursive approach to multiply two numbers.
 Let the two number be n digit numbers x and y, to keep the initial implementation simple, we assume both the numbers are n digits but can easily be extended to two numbers of different number of digits.
 
 
Let x and y be split into two $n/2$ digit numbers a, b and c,d respectively.

Thus 
$x = 10^{n/2}.a + b$ and $y = 10^ {n/2}.c + d$

$x.y = (10^ {n/2}.a + b) \times (10^ {n/2}.c + d) = 10^n.ab + 10^{n/2}.(a.d + b.c) + b.d $
 
 
Following code snippet ``rec_int_mult`` implements this recursive multiplication of two numbers. But first we will be defining a simple function which will take a number and pad it with leading 0 so that the length of the number if a power of 2 for simplicity in recursion

In [1]:
def pad_len_for_pow_2(num, min_len = None):
    inc_len = 1
    current_len = len(num)
    target_len = current_len if min_len is None else max(min_len, current_len)
    
    while inc_len < target_len:
        inc_len  = inc_len << 1
    
    if inc_len - current_len > 0:
        return '0' * (inc_len - current_len) + num
    else:
        return num
    
def prepare_inputs(in1, in2):
    max_len = max(len(in1), len(in2))
    return pad_len_for_pow_2(in1, max_len), pad_len_for_pow_2(in2, max_len)

In [2]:
def rec_int_mult(x, y):
    if len(x) == 1 and len(y) == 1:
        return int(x) * int(y)
    
    split = len(x) // 2
    a, b = x[0:split], x[split:]
    c, d = y[0:split], y[split:]
    # We cheat a bit here by multiplying by powers of 10 which can be implemented purely as addition
    return (10 ** len(x)) * rec_int_mult(a, c) + \
            (10 ** split) * (rec_int_mult(a, d) +\
                             rec_int_mult(b, c)) + rec_int_mult(b, d)

In [3]:
in1, in2 = prepare_inputs('5678', '1234')
rec_int_mult(in1, in2)

7006652


The above code snippet involves performing 4 multiplications of smaller digit numbers. Gauss instead worked out a way to replace these 4 multiplications with 3 multiplications and some additions.

Essentially $(a.d + b.c)$ be replaced with $(a + b).(c + d) - a.c - b.d$ thus reducing the multiplications to the following three multiplications

- $a.c$
- $b.d$
- $(a + b).(c + d)$

Essentially we reduce 4 recursive calls with 3 as seen in the following code snippet which is generic enough to accept numbers with not necessarily same number of digits

In [4]:
def karatsuba(x, y):
    if len(x) == 1 and len(y) == 1:
        return int(x) * int(y)
    split = len(x) // 2
    a, b = x[0:split], x[split:]
    c, d = y[0:split], y[split:]
    ac = karatsuba(a, c)
    bd = karatsuba(b, d)
    aplusb = str(int(a) + int(b))
    cplusd = str(int(c) + int(d))
    aplusb_times_cplusd  = karatsuba(*prepare_inputs(aplusb, cplusd))
    return (10 ** len(x)) *  ac + (10 ** split) * (aplusb_times_cplusd - ac - bd) + bd
    
    

In [5]:
in1, in2 = prepare_inputs('5678', '1234')
karatsuba(in1, in2)

7006652


The above result is same as the result we get from the conventional approach. 
Following two cells demonstrate execute the test case and the challenge problem given at [this](http://theory.stanford.edu/~tim/algorithmsilluminated.html) URL.

In [6]:
in1, in2 = prepare_inputs('99999', '9999')
karatsuba(in1, in2)

999890001

In [7]:
in1, in2 = prepare_inputs('3141592653589793238462643383279502884197169399375105820974944592',
                            '2718281828459045235360287471352662497757247093699959574966967627')
karatsuba(in1, in2)

8539734222673567065463550869546574495034888535765114961879601127067743044893204848617875072216249073013374895871952806582723184


By exactly how much Karatsuba multiplication is better than the conventional approach and is the divide and conquer approach in ``rec_int_mult`` any better than the conventional approach? We will look at answering this later after we finish the $4^{th}$ chapter where we explore the Master Method.

---

## Merge Sort

We will now analyse one of the most widely used and an efficient sorting algorithm, *Merge Sort*

---

Following implementation is an example of Insertion sort which requires 2 passes of the array giving it $O(n^2)$ complexity

In [8]:
def insertion(inp):
    res = [inp[i] for i in range(len(inp))]
    for i in range(len(res)):
        for j in range(i + 1, len(res)):
            if res[j] < res[i]:
                t = res[i]
                res[i] = res[j]
                res[j] = t                
            
    return res

In [9]:
insertion([9, 7, 1, 2, 4, 6, 3, 8, 5])

[1, 2, 3, 4, 5, 6, 7, 8, 9]


Quadratic time is not desirable for large arrays and we would ideally want to have an algorithm which have complexity between $O(n)$ and $O(n^2)$.

With this goal in mind, let us see what merge sort does and then later derive it's time complexity.

The following picture shows how merge sort works

![MergeSort](MergeSortIllustration.png)

Following three operations is what we have in Merge Sort

- Split the input array in two, this is usually done by finding the middle index of the input array
- Recursively call sort on these two splits, this operation should return us two sorted arrays
- Merge the two sorted arrays in one array which would be sorted.


In the following code snippet we will implement merge sort. To keep the code simple and readable, we will not be making it efficient in terms of space. That is, we can allocate the entire array upfront and have the subproblems update a part of this preinitialized array.

In [10]:
def merge(sorted1, sorted2):
    # Merges the two sorted arrays into one sorted array in linear time
    res = []
    i = 0
    j = 0    
    for k in range(len(sorted1) + len(sorted2)):        
        if i == len(sorted1):
            res = res + [sorted2[ind] for ind in range(j, len(sorted2))]
            break
        if j == len(sorted2):
            res = res + [sorted1[ind] for ind in range(i, len(sorted1))]
            break
        
        if sorted1[i] < sorted2[j]:
            res.append(sorted1[i])
            i += 1
        else:
            res.append(sorted2[j])
            j += 1
    
    return res

def mergesort(inarray):
    if len(inarray) <= 1:
        return inarray
    
    mid = len(inarray) // 2
    l = mergesort(inarray[:mid])
    r = mergesort(inarray[mid:])
    return merge(l, r)

In [11]:
mergesort([9, 7, 1, 2, 4, 6, 3, 8, 5])

[1, 2, 3, 4, 5, 6, 7, 8, 9]


Again, as mentioned previously the above implementation is not efficient in terms of space as each recursive call generates a new list which is then discarted. Nevertheless it is succinct and is a good translation of the pseudocode to a working code.

We are more interested in the running time complexity of the Merge sort which we will analyze now. Following is the psuedo code for the merge operation we implemented above

![MergePseudocode](MergePseudocode.png)

We see there are 2 operations for initilizing the variables i and j, and then the loop that runs n times.
In the loop we peform the following

- Comparison C[i] < D[j] 
- Assignment B[k] := C[i]  or B[k] := D[j]
- Increment of variable i or j
- Increment of loop variable k

This we have a total of $4n + 2$ operations. We will make this a total of $6n$ since $6n > 4n + 2$

The number $6n$ is just an estimate and doesn't really consider a programming language's implementation. For example, the Python implementation of merge we have is strkingly similar to the the merge pseudo code and also implements some edge cases when all elements of one of the array are copied over. For the sake of analysis $6n$ is a reasonable estimate.

We thus have the following Lemma

*For a pair of sorted input array of size $l/2$, merge function will merge them in one sorted list in no more than $6l$ operations*

Now that we have an upper bound on the running time of Merge, let us see how many operations are required for entire merge sort call.


